In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Apagar tabela Silver da Base de Dados
spark.sql(
"""
drop table if exists nba.sl_teams
"""
)

In [ ]:
#Mostrar que a tabela foi eliminada
spark.sql(
"""
show tables from nba
"""
).show()

In [ ]:
#Apagar ficheiros da tabela no HDFS
#Visto que estes continuam lá mesmo apos apagar tabela na DB
from os import PathLike
from hdfs import InsecureClient

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

delete_path = "hdfs://hdfs-nn:9000/projeto/nba/sl_teams"
client.delete(delete_path, True)

In [ ]:
#Carregar tabela de bronze para dataframe
table_location = "hdfs://hdfs-nn:9000/projeto/nba/bz_teams"
df = spark.read.format("delta").load(table_location)
df.toPandas()

In [ ]:
#Delete Colunas
from pyspark.sql.functions import col

df2 = df.drop("Coaches","Rel_ORtg","Rel_DRtg","W/L%","Rel_Pace")

In [ ]:
df2.toPandas()

In [ ]:
#Rename Colunas
df3 = df2.withColumnRenamed("Season","Ano") \
    .withColumnRenamed("Lg","Liga")\
    .withColumnRenamed("Team","Equipa")\
    .withColumnRenamed("W","Vitorias")\
    .withColumnRenamed("L","Derrotas")\
    .withColumnRenamed("Finish","Lugar")\
    .withColumnRenamed("TopWs","MelhorJogador")

df3.printSchema()

In [ ]:
df3.toPandas()

In [ ]:
#Eliminar os * na coluna Equipa para ficar com os nomes uniformes
import pyspark.sql.functions as f
from pyspark.sql.functions import trim,ltrim,rtrim

df4 = df3.withColumn(
    "Equipa",
    f.regexp_replace(f.col("Equipa"), "[*]", "")
)

df4 = df4.withColumn("Equipa",ltrim("Equipa"))
df4.toPandas()

In [ ]:
#Reordenar Colunas
df5 = df4.select("Liga","Ano","Equipa","Lugar","Playoffs","Vitorias","Derrotas","SRS","Pace","ORtg","DRtg","MelhorJogador")
df5.toPandas()

In [ ]:
#Criar coluna com total de jogos
df6= df5.withColumn(    
    "Jogos",
    col("Vitorias")+col("Derrotas")
)

df6.toPandas()

In [ ]:
#Coluna %Vitorias
df7 = df6.withColumn(
    "Vitorias%",
    col("Vitorias")/col("Jogos")*100)

df7.toPandas()

In [ ]:
#Separar valores da coluna Ano
#2008-09 para 2008

import pyspark.sql.functions as f

df8 = df7.withColumn("2ano", f.split(df7['Ano'], '\-')[1]).\
           withColumn("1ano", f.split(df7['Ano'], '\-')[0])

df8.toPandas()

In [ ]:
#mudar o data type da coluna 1ano 
from pyspark.sql.types import IntegerType

df9 = df8.withColumn("1ano", df8["1ano"].cast(IntegerType()))

df9.printSchema()

In [ ]:
#Adicionar 1 ano a coluna "2ano"
#Exemplo: 2008 vai ser 2008+1 para ficar 2009
df10 = df9.withColumn(
    "1ano",
    col("1ano") + 1)

df10.toPandas()

In [ ]:
#Apagar as colunas "Ano" e "2ano"
df11 = df10.drop("Ano","2ano")

#Renomear a coluna "1ano" para "Ano"
df12 = df11.withColumnRenamed("1ano","Ano")

df12.toPandas()

In [ ]:
#Eliminar NULL na coluna dos Playoffs
#Os null vao ser substituidos por "" (vazio)
#Pq na funcao de alterar valores das colunas nao esta a reconhecer os valores NULL
#Mesmo usando == NONE que e o equivalente de NULL em Python
dfplay = df12.na.fill("",["Playoffs"])
dfplay.toPandas()

In [ ]:
#substituir valores na coluna PlayOffs
from pyspark.sql.functions import when
df13 = dfplay.withColumn("Playoffs", when(col("Playoffs") == "","N/Qualificou") \
    .when(col("Playoffs") == "Won Finals","Venceu Final") \
    .when(col("Playoffs") == "Lost Finals","Perdeu Final") \
    .otherwise("Eliminado"))

df13.toPandas()

In [ ]:
#Carregar tabela das abreviacoes para dar join
#tabela de bronze para dataframe
location1 = "hdfs://hdfs-nn:9000/projeto/nba/bz_abrev"
df_abrev = spark.read.format("delta").load(location1)
df_abrev.toPandas()

In [ ]:
#Renomear Colunas para nao haver conflitos de nomes
#Tinha dado erro sem fazer isto
#porque iamos ter colunas com nomes repetidos e nos drops e renames ia alterar todas as colunas
df_abrev1 = df_abrev.withColumnRenamed("equipa","EquipaDef")\
    .withColumnRenamed("liga","LigaDef")\
    .withColumnRenamed("sigla","Sigla")

df_abrev1.printSchema()

In [ ]:
#Join
#Entre abreviacoes e equipas
cond = [df_abrev1.EquipaDef == df13.Equipa, df_abrev1.LigaDef == df13.Liga]
df14= df_abrev1.join(df13,cond, 'fullouter')
df14.toPandas()

In [ ]:
df14.show(1662)

In [ ]:
#Eliminar NULL nas 2 coluna das Equipas
dfnew = df14.na.fill("",["Equipa"]) \
    .na.fill("",["Equipadef"])

#Complementar valores de uma coluna para a outra
#se uma coluna nao tiver o nome, vai buscar à outra
dfnew = dfnew.withColumn(
    "Equipa",
    when( (col("Equipa") == ""), col("EquipaDef"))\
    .otherwise(col("Equipa"))
)

dfnew = dfnew.withColumn(
    "EquipaDef",
    when( (col("EquipaDef") == ""), col("Equipa"))\
    .otherwise(col("EquipaDef"))
)

#Eliminar NULL nas seguintes colunas
dfnew = dfnew.na.fill("",["Liga"])
dfnew = dfnew.na.fill("",["Lugar"])

#Complementar as 2 colunas para ficarmos com os valores todos e desaparecerem os vazios
dfnew = dfnew.withColumn(
    "Liga",
    when( col("Liga") == "", col("LigaDef"))\
    .otherwise(col("Liga"))
)

dfnew = dfnew.withColumn(
    "Lugar",
    when( col("Lugar") == "", "-")\
    .otherwise(col("Lugar"))
)

#Substituir Valores que nao existiam na coluna Sigla
dfnew = dfnew.withColumn(
    "Sigla",
    when(col("EquipaDef") == "Fort Wayne Pistons","FTW")\
    .when(col("EquipaDef") == "Seattle SuperSonics","SEA")\
    .when(col("EquipaDef") == "New York Knicks","NYK")\
    .when(col("EquipaDef") == "Portland Trail Blazers","POR")\
    .when(col("EquipaDef") == "Denver Rockets","DEN")\
    .when(col("EquipaDef") == "Brooklyn Nets","BKN")\
    .when(col("EquipaDef") == "Washington Bullets","WAS")\
    .when(col("EquipaDef") == "Oklahoma City Thunder","OKC")\
    .when(col("EquipaDef") == "Washington Wizards","WSB")\
    .otherwise(col("Sigla"))
)

dfnew.select("Sigla","EquipaDef","LigaDef","Liga","Equipa").show(1662)

In [ ]:
#Apagar as colunas "Equipa", "Liga"
df15 = dfnew.drop("Equipa","Liga")

df15.toPandas()

In [ ]:
#Renomear a coluna
df16 = df15.withColumnRenamed("EquipaDef","Equipa")\
    .withColumnRenamed("LigaDef","Liga")

df16.toPandas()

In [ ]:
df16.printSchema()

In [35]:
#Mudar o tipo da coluna Ano de INT para STRING
from pyspark.sql.types import StringType
df17 = df16.withColumn("Ano", df16["Ano"].cast(StringType()))
#df17.toPandas()

In [ ]:
df17.select("Equipa","Sigla").show(1662)

In [36]:
#Correções para ficar igual com a tabela dos players
#Uniformizar siglas
df18 = df17.withColumn(
    "Sigla",
    when( (col("Equipa") == "Golden State Warriors"), "GSW")\
    .when( (col("Equipa") == "Utah Jazz"), "NOJ")\
    .when( (col("Equipa") == "Oklahoma City Thunder"), "OKC")\
    .when( (col("Equipa") == "Phoenix Suns"), "PHO")\
    .when( (col("Equipa") == "Denver Nuggets"), "DEN")\
    .when( (col("Equipa") == "Washington Wizards"), "WSB")\
    .when( (col("Equipa") == "San Antonio Spurs"), "SAS")\
    .otherwise(col("Sigla"))
)

df18.select("Equipa", "Sigla").show(1662)

+--------------------+-----+
|              Equipa|Sigla|
+--------------------+-----+
|      Anaheim Amigos|  ANA|
|     Baltimore Claws|  BAC|
|   California Dreams|  CAD|
|Cleveland Allmen ...|  CLA|
|    Cleveland Rebels|  CLR|
|  Evansville Agogans|  EVN|
|  Fort Wayne Pistons|  FTW|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston Rockets|  HOU|
|     Houston 

In [37]:
#Alterar valor da coluna das siglas
dfsigla = df18.withColumn(
    "Sigla",
    when(col("Sigla") == "GSW", "GST")\
    .when(col("Sigla") == "SAS", "SAN")\
    .when(col("Sigla") == "NOJ", "UTA")\
    .when(col("Sigla") == "PHO", "PHX")\
    .otherwise(col("Sigla"))
)

dfsigla.select("Sigla","Equipa").show(1662)

+-----+--------------------+
|Sigla|              Equipa|
+-----+--------------------+
|  ANA|      Anaheim Amigos|
|  BAC|     Baltimore Claws|
|  CAD|   California Dreams|
|  CLA|Cleveland Allmen ...|
|  CLR|    Cleveland Rebels|
|  EVN|  Evansville Agogans|
|  FTW|  Fort Wayne Pistons|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Houston Rockets|
|  HOU|     Ho

In [38]:
#Mais correções
dfsigla = dfsigla.withColumn(
    "Sigla",
    when(col("Equipa") == "Indiana Pacers", "IND")\
    .when(col("Equipa") == "New Orleans Hornets", "NOH")\
    .when(col("Equipa") == "Boston Celtics", "BOS")\
    .when(col("Equipa") == "New Orleans Pelicans", "NYP")\
    .when(col("Equipa") == "New York Nets", "NYN")\
    .otherwise(col("Sigla"))
)

In [40]:
#Criacao da coluna das Conferencias
#Associar uma equipa à respetiva conferencia em que joga
df19 = dfsigla.withColumn(
    "Conferencia",
    when( (col("Sigla") == "GSW"), "Oeste")\
    .when( (col("Sigla") == "HOU"), "Oeste")\
    .when( (col("Sigla") == "CIN"), "Este")\
    .when( (col("Sigla") == "LAC"), "Oeste")\
    .when( (col("Sigla") == "DET"), "Este")\
    .when( (col("Sigla") == "GSW"), "Oeste")\
    .when( (col("Sigla") == "DNN"), "Oeste")\
    .when( (col("Sigla") == "BAL"), "Este")\
    .when( (col("Sigla") == "BLB"), "Este")\
    .when( (col("Sigla") == "SYR"), "Este")\
    .when( (col("Sigla") == "PHW"), "Oeste")\
    .when( (col("Sigla") == "IND"), "Este")\
    .when( (col("Sigla") == "SAS"), "Oeste")\
    .when( (col("Sigla") == "KCO"), "Oeste")\
    .when( (col("Sigla") == "PHO"), "Oeste")\
    .when( (col("Sigla") == "NOR"), "Oeste")\
    .when( (col("Sigla") == "SFW"), "Oeste")\
    .when( (col("Sigla") == "ROC"), "Oeste")\
    .when( (col("Sigla") == "NOJ"), "Oeste")\
    .when( (col("Sigla") == "MIL"), "Este")\
    .when( (col("Sigla") == "KCK"), "Oeste")\
    .when( (col("Sigla") == "MIN"), "Oeste")\
    .when( (col("Sigla") == "CLE"), "Este")\
    .when( (col("Sigla") == "BOS"), "Este")\
    .when( (col("Sigla") == "LAL"), "Oeste")\
    .when( (col("Sigla") == "TOR"), "Este")\
    .when( (col("Sigla") == "SDR"), "Oeste")\
    .when( (col("Sigla") == "MEG"), "Oeste")\
    .when( (col("Sigla") == "MIA"), "Este")\
    .when( (col("Sigla") == "ORL"), "Este")\
    .when( (col("Sigla") == "CHA"), "Este")\
    .when( (col("Sigla") == "VAN"), "Oeste")\
    .when( (col("Sigla") == "DAL"), "Oeste")\
    .when( (col("Sigla") == "NJN"), "Este")\
    .when( (col("Sigla") == "STL"), "Oeste")\
    .when( (col("Sigla") == "ATL"), "Este")\
    .when( (col("Sigla") == "CHN"), "Este")\
    .when( (col("Sigla") == "MLH"), "Este")\
    .when( (col("Sigla") == "MPL"), "Oeste")\
    .when( (col("Sigla") == "SDC"), "Oeste")\
    .when( (col("Sigla") == "PHI"), "Este")\
    .when( (col("Sigla") == "NOK"), "Oeste")\
    .when( (col("Sigla") == "CHI"), "Este")\
    .when( (col("Sigla") == "BUF"), "Este")\
    .when( (col("Sigla") == "SAC"), "Oeste")\
    .when( (col("Sigla") == "GST"), "Oeste")\
    .when( (col("Sigla") == "DEN"), "Oeste")\
    .when( (col("Sigla") == "WSB"), "Este")\
    .when( (col("Sigla") == "NOH"), "Oeste")\
    .when( (col("Sigla") == "NYN"), "Este")\
    .when( (col("Sigla") == "SAN"), "Oeste")\
    .when( (col("Sigla") == "PHX"), "Oeste")\
    .when( (col("Sigla") == "FTW"), "Este")\
    .when( (col("Sigla") == "WAS"), "Este")\
    .when( (col("Sigla") == "UTA"), "Oeste")\
    .when( (col("Sigla") == "IND"), "Este")\
    .when( (col("Sigla") == "SEA"), "Oeste")\
    .when( (col("Sigla") == "OKC"), "Oeste")\
    .when( (col("Sigla") == "BOS"), "Este")\
    .when( (col("Sigla") == "NYK"), "Este")\
    .when( (col("Sigla") == "POR"), "Oeste")\
    .when( (col("Sigla") == "DEN"), "Oeste")\
    .when( (col("Sigla") == "NYP"), "Este")\
    .when( (col("Sigla") == "BKN"), "Este")\
    .otherwise("NaN")
)

df19.select("Equipa", "Sigla", "Conferencia").show(1662)

+--------------------+-----+-----------+
|              Equipa|Sigla|Conferencia|
+--------------------+-----+-----------+
|      Anaheim Amigos|  ANA|        NaN|
|     Baltimore Claws|  BAC|        NaN|
|   California Dreams|  CAD|        NaN|
|Cleveland Allmen ...|  CLA|        NaN|
|    Cleveland Rebels|  CLR|        NaN|
|  Evansville Agogans|  EVN|        NaN|
|  Fort Wayne Pistons|  FTW|       Este|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Rockets|  HOU|      Oeste|
|     Houston Ro

In [44]:
df19.toPandas()

Ano   Liga Sigla                      Equipa Conferencia Lugar  \
0     None  OUTRA   ANA              Anaheim Amigos         NaN  None   
1     None  OUTRA   BAC             Baltimore Claws         NaN  None   
2     None  OUTRA   CAD           California Dreams         NaN  None   
3     None  OUTRA   CLA  Cleveland Allmen Transfers         NaN  None   
4     None  OUTRA   CLR            Cleveland Rebels         NaN  None   
...    ...    ...   ...                         ...         ...   ...   
1657  1990    NBA   SAC            Sacramento Kings       Oeste     7   
1658  1989    NBA   SAC            Sacramento Kings       Oeste     6   
1659  1988    NBA   SAC            Sacramento Kings       Oeste     6   
1660  1987    NBA   SAC            Sacramento Kings       Oeste     5   
1661  1986    NBA   SAC            Sacramento Kings       Oeste     5   

          Playoffs  Jogos  Vitorias  Derrotas  Vitorias%      MelhorJogador  \
0             None    NaN       NaN       NaN        NaN               None   
1             None    NaN       NaN       NaN        NaN               None   
2             None    NaN       NaN       NaN        NaN               None   
3             None    NaN       NaN       NaN        NaN               None   
4             None    NaN       NaN       NaN        NaN               None   
...            ...    ...       ...       ...        ...                ...   
1657  N/Qualificou   82.0      23.0      59.0  28.048780    R. McCray (7.9)   
1658  N/Qualificou   82.0      27.0      55.0  32.926829  H. Pressley (4.8)   
1659  N/Qualificou   82.0      24.0      58.0  29.268293    O. Thorpe (8.1)   
1660  N/Qualificou   82.0      29.0      53.0  35.365854    O. Thorpe (9.0)   
1661     Eliminado   82.0      37.0      45.0  45.121951     R. Theus (6.4)   

       SRS   Pace   ORtg   DRtg  
0      NaN    NaN    NaN    NaN  
1      NaN    NaN    NaN    NaN  
2      NaN    NaN    NaN    NaN  
3      NaN    NaN    NaN    NaN  
4      NaN    NaN    NaN    NaN  
...    ...    ...    ...    ...  
1657 -4.41   96.7  104.8  110.0  
1658 -5.35  100.5  104.5  110.0  
1659 -5.84  100.9  106.5  112.1  
1660 -3.34  101.3  108.9  112.1  
1661 -3.19  102.0  106.2  109.2  

[1662 rows x 16 columns]

In [41]:
#Reordenar
df19 = df19.select("Ano","Liga","Sigla","Equipa","Conferencia","Lugar","Playoffs","Jogos","Vitorias","Derrotas","Vitorias%","MelhorJogador","SRS","Pace","ORtg","DRtg")
df19.printSchema()

root
 |-- Ano: string (nullable = true)
 |-- Liga: string (nullable = true)
 |-- Sigla: string (nullable = true)
 |-- Equipa: string (nullable = false)
 |-- Conferencia: string (nullable = false)
 |-- Lugar: string (nullable = true)
 |-- Playoffs: string (nullable = true)
 |-- Jogos: integer (nullable = true)
 |-- Vitorias: integer (nullable = true)
 |-- Derrotas: integer (nullable = true)
 |-- Vitorias%: double (nullable = true)
 |-- MelhorJogador: string (nullable = true)
 |-- SRS: double (nullable = true)
 |-- Pace: double (nullable = true)
 |-- ORtg: double (nullable = true)
 |-- DRtg: double (nullable = true)



In [ ]:
#Create Silver Table in DataBase
spark.sql(
    """
    CREATE EXTERNAL TABLE nba.sl_teams (
        Sigla STRING, Equipa STRING, Liga STRING, Vitorias INT, Derrotas INT,
        Classificacao INT, SRS DOUBLE, Pace DOUBLE, ORtg DOUBLE, DRtg DOUBLE, Playoffs STRING,
        MelhorJogador STRING, Jogos INT, Vitorias_Perc DOUBLE
    )
    USING DELTA
    PARTITIONED BY(
    Ano INT
    )
    TBLPROPERTIES (
    'skip.header.line.count'='1'
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/sl_teams'
    """
)

In [42]:
#Guardar o ultimo DataFrama (df19) na hivetable de silver

df19.select("*")\
    .write\
    .mode("overwrite")\
    .option("overwriteSchema","true")\
    .partitionBy("Ano") \
    .format("delta")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/sl_teams")

22/01/03 15:05:18 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [45]:
#Descrever historico da tabela
spark.sql(
"""
DESCRIBE HISTORY nba.sl_players
"""
).show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      5|2022-01-03 15:15:...|  null|    null|       WRITE|{mode -> Overwrit...|null|    null|     null|          4|          null|        false|{numFiles -> 1484...|        null|
|      4|2021-12-05 17:09:...|  null|    null|       WRITE|{mode -> Overwrit...|null|    null|     null|          3|          null|        false|{numFiles -> 141,...|        null|
|      3|2021-12-05 15:29:...|  null|    null|       WRITE|{mode -> Overwrit...|null|    null|     n

In [ ]:
spark.sql(
"""
show tables from nba
"""
).show()